In [1]:
import os
import cv2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, LSTM, Dense, TimeDistributed, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [2]:
metadata_path = r'C:\Projects\DeepfakeDetection\DeepFakeDetection_Dataset\metadata.csv'
metadata = pd.read_csv(metadata_path)


In [3]:
video_dir = r'C:\Projects\DeepfakeDetection\DeepFakeDetection_Dataset\data'
metadata['video_path'] = metadata['video_path'].apply(lambda x: os.path.join(video_dir, x))
metadata['label'] = metadata['label'].apply(lambda x: 1 if x == 'REAL' else 0)


In [4]:
metadata

,video_path,label
0,C:\Projects\DeepfakeDetection\DeepFakeDetectio...,1
1,C:\Projects\DeepfakeDetection\DeepFakeDetectio...,0
2,C:\Projects\DeepfakeDetection\DeepFakeDetectio...,1
3,C:\Projects\DeepfakeDetection\DeepFakeDetectio...,0
4,C:\Projects\DeepfakeDetection\DeepFakeDetectio...,1
...,...,...
1995,C:\Projects\DeepfakeDetection\DeepFakeDetectio...,0
1996,C:\Projects\DeepfakeDetection\DeepFakeDetectio...,1
1997,C:\Projects\DeepfakeDetection\DeepFakeDetectio...,0
1998,C:\Projects\DeepfakeDetection\DeepFakeDetectio...,1


In [5]:
def extract_frames(video_path, num_frames=15):
    cap = cv2.VideoCapture(video_path)
    frames = []
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    skip = max(int(frame_count/num_frames), 1)
    
    for i in range(0, frame_count, skip):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i)
        ret, frame = cap.read()
        if ret:
            frame = cv2.resize(frame, (224, 224))
            frames.append(frame)
        if len(frames) == num_frames:
            break
    cap.release()
    return np.array(frames)


In [6]:
video_data = []
labels = []

for idx, row in metadata.iterrows():
    frames = extract_frames(row['video_path'])
    if len(frames) == 15:
        video_data.append(frames)
        labels.append(row['label'])

video_data = np.array(video_data)
labels = np.array(labels)


In [7]:
labels

array([1, 0, 1, ..., 0, 1, 0])

In [8]:
X_train, X_test, y_train, y_test = train_test_split(video_data, labels, test_size=0.2, random_state=42, stratify=labels)


In [9]:
model = Sequential()
model.add(TimeDistributed(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3))))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
model.add(TimeDistributed(Conv2D(64, (3, 3), activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
model.add(TimeDistributed(Flatten()))

model.add(LSTM(64, return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])


C:\Users\abish\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=8)


Epoch 1/20
199/199 ━━━━━━━━━━━━━━━━━━━━ 5871s 29s/step - accuracy: 0.5260 - loss: 0.7127 - val_accuracy: 0.5000 - val_loss: 0.6943
Epoch 2/20
199/199 ━━━━━━━━━━━━━━━━━━━━ 8344s 42s/step - accuracy: 0.4783 - loss: 0.7074 - val_accuracy: 0.5000 - val_loss: 0.6943
Epoch 3/20
199/199 ━━━━━━━━━━━━━━━━━━━━ 5564s 28s/step - accuracy: 0.4972 - loss: 0.6988 - val_accuracy: 0.5000 - val_loss: 0.6932
Epoch 4/20
199/199 ━━━━━━━━━━━━━━━━━━━━ 6111s 31s/step - accuracy: 0.5243 - loss: 0.6979 - val_accuracy: 0.5000 - val_loss: 0.6938
Epoch 5/20
199/199 ━━━━━━━━━━━━━━━━━━━━ 8815s 44s/step - accuracy: 0.4984 - loss: 0.6932 - val_accuracy: 0.5000 - val_loss: 0.6944
Epoch 6/20
199/199 ━━━━━━━━━━━━━━━━━━━━ 6263s 32s/step - accuracy: 0.5033 - loss: 0.6993 - val_accuracy: 0.5000 - val_loss: 0.6946
Epoch 7/20
199/199 ━━━━━━━━━━━━━━━━━━━━ 5584s 28s/step - accuracy: 0.4962 - loss: 0.6993 - val_accuracy: 0.5000 - val_loss: 0.6932
Epoch 8/20
199/199 ━━━━━━━━━━━━━━━━━━━━ 5235s 26s/step - accuracy: 0.4847 - loss: 0